In [1]:
from ultralytics import YOLO
from utils import *
from IPython.display import Image, display, Video
from roboflow import Roboflow

In [2]:
# rf = Roboflow(api_key="nz2w1UFnyFrM7e73WOzh")
# project = rf.workspace("nyu-figsb").project("basketballdetection-cki6r")
# dataset = project.version(27).download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.228, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to basketballDetection-27 in yolov8::  41%|████      | 7226/17736 [05:04<07:22, 23.76it/s]  


KeyboardInterrupt: 

In [2]:
model = YOLO("runs/detect/train12/weights/last.pt")
# model = YOLO("runs/detect/train7/weights/last.pt")
model.train(data = "basketballDetection-27/data.yaml", 
            epochs = 350, 
            batch = 64, 
            # imgsz = 640, 
            save_period=20,
            # lr0 = 0.001,
            lrf = 0.005,
            workers = 10,
            device = [0, 1],
            patience = 30,
            resume = True,
            mosaic = 0.3
            )

New https://pypi.org/project/ultralytics/8.0.231 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.228 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (Quadro RTX 8000, 45548MiB)
                                                             CUDA:1 (Quadro RTX 8000, 45548MiB)
engine/trainer: task=detect, mode=train, model=runs/detect/train12/weights/last.pt, data=basketballDetection-27/data.yaml, epochs=350, time=None, patience=30, batch=64, imgsz=640, save=True, save_period=20, cache=False, device=[0, 1], workers=10, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=Fa

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.0.228 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (Quadro RTX 8000, 45548MiB)
                                                             CUDA:1 (Quadro RTX 8000, 45548MiB)


Traceback (most recent call last):
  File "/home/zw2688/.config/Ultralytics/DDP/_temp__wlo2e46139817031073648.py", line 12, in <module>
    results = trainer.train()
  File "/scratch/zw2688/conda_env/yolonas/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 190, in train
    self._do_train(world_size)
  File "/scratch/zw2688/conda_env/yolonas/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 290, in _do_train
    self._setup_train(world_size)
  File "/scratch/zw2688/conda_env/yolonas/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 217, in _setup_train
    ckpt = self.setup_model()
  File "/scratch/zw2688/conda_env/yolonas/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 476, in setup_model
    weights, ckpt = attempt_load_one_weight(model)
  File "/scratch/zw2688/conda_env/yolonas/lib/python3.10/site-packages/ultralytics/nn/tasks.py", line 628, in attempt_load_one_weight
    ckpt, weight = torch_safe_load(weight)  # loa

TensorBoard: Start with 'tensorboard --logdir runs/detect/train12', view at http://localhost:6006/


wandb: Currently logged in as: twowzt. Use `wandb login --relogin` to force relogin
[2023-12-30 11:49:55,807] torch.distributed.elastic.multiprocessing.api: [WARNING] Sending process 1662695 closing signal SIGTERM
[2023-12-30 11:49:55,871] torch.distributed.elastic.multiprocessing.api: [ERROR] failed (exitcode: 1) local_rank: 1 (pid: 1662696) of binary: /scratch/zw2688/conda_env/yolonas/bin/python
Traceback (most recent call last):
  File "/scratch/zw2688/conda_env/yolonas/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/scratch/zw2688/conda_env/yolonas/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/scratch/zw2688/conda_env/yolonas/lib/python3.10/site-packages/torch/distributed/run.py", line 810, in <module>
    main()
  File "/scratch/zw2688/conda_env/yolonas/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 346, in wrapper
    return f

CalledProcessError: Command '['/scratch/zw2688/conda_env/yolonas/bin/python', '-m', 'torch.distributed.run', '--nproc_per_node', '2', '--master_port', '47795', '/home/zw2688/.config/Ultralytics/DDP/_temp__wlo2e46139817031073648.py']' returned non-zero exit status 1.